In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
%matplotlib inline

In [3]:
data = pd.read_csv('./data/train.csv')

#### Функции очистки данных

In [4]:
def clear_healthcare(ds):
    return ds.drop(columns=['Healthcare_1'], axis=1)

In [5]:
def clear_houseyear(ds):
    rows_to_clear = ds.loc[(ds.HouseYear < 1900) | (ds.HouseYear > 2020)]
    df_valid = ds.loc[(ds.HouseYear >= 1900) | (ds.HouseYear <= 2020)]
    max_year = df_valid.HouseYear.max()
    min_year = df_valid.HouseYear.min()

    for index, row in rows_to_clear.iterrows():
        sqr_delta = row.Square / row.Rooms
        year = ds.loc[(df_valid.Id != row.Id) & 
                        (df_valid.DistrictId == row.DistrictId) & 
                        (df_valid.Rooms == row.Rooms) & 
                        (df_valid.Square <= row.Square + sqr_delta) & (df_valid.Square >= row.Square - sqr_delta)
                       ].HouseYear.mean()

        ds.loc[index, 'HouseYear'] = np.clip(np.int64(year), min_year, max_year)
    
    return ds

In [6]:
data = clear_healthcare(data)
data = clear_houseyear(data)

In [7]:
data.loc[(data.Id == 10814) | (data.Id == 11607)]

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Helthcare_2,Shops_1,Shops_2,Price
1497,10814,109,1.0,37.265070,20.239714,9.0,9,12.0,1987,0.136330,B,B,30,6141,10,3,6,B,254084.534396
4189,11607,147,2.0,44.791836,28.360393,5.0,4,9.0,1967,0.319809,B,B,25,4756,16,5,8,B,243028.603096
